<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/AllEmoInt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from google.colab import  drive
drive.mount('/content/drive')
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
device_name = tf.test.gpu_device_name()
print(device_name)
df = pd.read_csv('/content/drive/My Drive/IA/Clean_Tweets_data.csv')
def remove_stop(text):
  sw = stopwords.words("english")
  sw.append('br')
  clean = " ".join([x for x in text.split() if x not in sw])
  return clean
df['Tweet'] = df['Tweet'].apply(lambda tweet : remove_stop(tweet))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/device:GPU:0


In [ ]:
category_dict = {"anger":[1,0,0,0],"sadness":[0,1,0,0],"fear":[0,0,1,0],"joy":[0,0,0,1]}
data_tweet = [x.lower().split() for x in df['Tweet']]
data_cat = np.array([category_dict[x] for x in df['Emotion']])

In [ ]:
print(data_tweet[:5])
print(data_cat[:5])

[['like', 'i', 'cannot', '12', 'hours', 'worth', 'standing', 'calorie', 'burning', '3', 'hours', 'loudly', 'crying', 'faceloudly', 'crying', 'faceloudly', 'crying', 'faceloudly', 'crying', 'face', '1stworldprobs4vs'], ['darkest', 'nights', 'bright', 'solemn', 'faces', 'lights', 'smile', 'believe', 'i'], ['true', 'depression', 'fans', 'get', 'one', 'face', 'tears', 'joyface', 'tears', 'joy'], ['awe', 'yay', 'thank', 'good', 'i', 'worried'], ['on', 'way', 'rejoicing', 'pulpit', 'park', 'city', 'baptist', 'church', 'surrey', 'bc', 'midweek', 'service']]
[[1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]


In [ ]:
# Parameters
Min_count = 0
Embedding_size = 100
Window_size = 5
Negative_sampling = 00

In [ ]:
w2v_sg = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=1)
w2v_sg.build_vocab(data_tweet)
w2v_sg.train(data_tweet, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=0)
w2v_cbow.build_vocab(data_tweet)
w2v_cbow.train(data_tweet, total_examples=w2v_cbow.corpus_count, epochs=5)

(350721, 369570)

In [ ]:
vocab_sg = w2v_sg.wv.vocab
vocab_sg = [x for x in vocab_sg]
vocab_cbow = w2v_cbow.wv.vocab
vocab_cbow = [x for x in vocab_cbow] 
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/glove/glove.twitter.27B.100d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [ ]:
data_tweet_vec_sg = np.array([np.array([w2v_sg.wv.get_vector(i) for i in x if i in vocab_sg]) for x in data_tweet])
data_tweet_vec_cbow = np.array([np.array([w2v_cbow.wv.get_vector(i) for i in x if i in vocab_cbow]) for x in data_tweet])
data_tweet_vec_glove = np.array([np.array([vocab_glove[i] for i in x if i in vocab_glove.keys()]) for x in data_tweet])
Max_input_size_glove = max([len(x) for x in data_tweet_vec_glove])
Max_input_size=max([len(x) for x in data_tweet_vec_sg]+[len(x) for x in data_tweet_vec_cbow])

In [ ]:
data_tweet_vec_sg = np.array([np.pad(x.flatten(),(0,Max_input_size*Embedding_size-len(x.flatten()))).reshape(Max_input_size,Embedding_size) for x in data_tweet_vec_sg])
data_tweet_vec_cbow = np.array([np.pad(x.flatten(),(0,Max_input_size*Embedding_size-len(x.flatten()))).reshape(Max_input_size,Embedding_size) for x in data_tweet_vec_cbow])
data_tweet_vec_glove = np.array([np.pad(x.flatten(),(0,Max_input_size_glove*100-len(x.flatten()))).reshape(Max_input_size_glove,100) for x in data_tweet_vec_glove])


In [ ]:
X_train_sg, X_test_sg, y_train_sg, y_test_sg = train_test_split(data_tweet_vec_sg, data_cat, test_size=0.30, random_state=42)
X_train_cbow, X_test_cbow, y_train_cbow, y_test_cbow = train_test_split(data_tweet_vec_cbow, data_cat, test_size=0.30, random_state=42)
X_train_glove, X_test_glove, y_train_glove, y_test_glove = train_test_split(data_tweet_vec_glove, data_cat, test_size=0.30, random_state=42)

In [ ]:
result_table = []

In [ ]:
model_sg = tf.keras.Sequential()
model_sg.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg.add(tf.keras.layers.LSTM(50))
model_sg.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg.fit(X_train_sg,y_train_sg,epochs=20,validation_split=0.2)
history = model_sg.evaluate(X_test_sg,y_test_sg)
print(history)
result_table+=[history[1]]
print("SG_LSTM")

Epoch 1/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3729 - accuracy: 0.3265 - val_loss: 1.3845 - val_accuracy: 0.2978
Epoch 2/20
125/125 [==============================] - 1s 5ms/step - loss: 1.3715 - accuracy: 0.3272 - val_loss: 1.3827 - val_accuracy: 0.2978
Epoch 3/20
125/125 [==============================] - 1s 5ms/step - loss: 1.3709 - accuracy: 0.3272 - val_loss: 1.3884 - val_accuracy: 0.2978
Epoch 4/20
125/125 [==============================] - 1s 6ms/step - loss: 1.3707 - accuracy: 0.3272 - val_loss: 1.3849 - val_accuracy: 0.2978
Epoch 5/20
125/125 [==============================] - 1s 5ms/step - loss: 1.3705 - accuracy: 0.3272 - val_loss: 1.3882 - val_accuracy: 0.2978
Epoch 6/20
125/125 [==============================] - 1s 5ms/step - loss: 1.3704 - accuracy: 0.3272 - val_loss: 1.3872 - val_accuracy: 0.2978
Epoch 7/20
125/125 [==============================] - 1s 6ms/step - loss: 1.3706 - accuracy: 0.3272 - val_loss: 1.3843 - val_accuracy: 0.2978
Epoch 

In [ ]:
model_sg_bi = tf.keras.Sequential()
model_sg_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_sg_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg_bi.fit(X_train_sg,y_train_sg,epochs=20,validation_split=0.2)
history = model_sg_bi.evaluate(X_test_sg,y_test_sg)
print(history)

result_table+=[history[1]]
print("SG_BI")

Epoch 1/20
125/125 [==============================] - 2s 14ms/step - loss: 1.3720 - accuracy: 0.3252 - val_loss: 1.3852 - val_accuracy: 0.2978
Epoch 2/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3706 - accuracy: 0.3272 - val_loss: 1.3834 - val_accuracy: 0.2978
Epoch 3/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3700 - accuracy: 0.3272 - val_loss: 1.3856 - val_accuracy: 0.2978
Epoch 4/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3692 - accuracy: 0.3272 - val_loss: 1.3814 - val_accuracy: 0.2978
Epoch 5/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3692 - accuracy: 0.3272 - val_loss: 1.3826 - val_accuracy: 0.2978
Epoch 6/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3692 - accuracy: 0.3272 - val_loss: 1.3816 - val_accuracy: 0.2978
Epoch 7/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3689 - accuracy: 0.3272 - val_loss: 1.3862 - val_accuracy: 0.2978
Epoch

In [ ]:
model_sg_rnn = tf.keras.Sequential()
model_sg_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_rnn.add(tf.keras.layers.SimpleRNN(50))
model_sg_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg_rnn.fit(X_train_sg,y_train_sg,epochs=20,validation_split=0.2)
history = model_sg_rnn.evaluate(X_test_sg,y_test_sg)
print(history)

result_table+=[history[1]]
print("SG_RNN")

Epoch 1/20
125/125 [==============================] - 4s 32ms/step - loss: 1.3756 - accuracy: 0.3214 - val_loss: 1.3857 - val_accuracy: 0.2978
Epoch 2/20
125/125 [==============================] - 4s 31ms/step - loss: 1.3665 - accuracy: 0.3232 - val_loss: 1.4006 - val_accuracy: 0.2928
Epoch 3/20
125/125 [==============================] - 4s 31ms/step - loss: 1.3642 - accuracy: 0.3249 - val_loss: 1.3887 - val_accuracy: 0.2877
Epoch 4/20
125/125 [==============================] - 4s 31ms/step - loss: 1.3628 - accuracy: 0.3297 - val_loss: 1.3823 - val_accuracy: 0.2978
Epoch 5/20
125/125 [==============================] - 4s 32ms/step - loss: 1.3599 - accuracy: 0.3292 - val_loss: 1.3913 - val_accuracy: 0.2847
Epoch 6/20
125/125 [==============================] - 4s 32ms/step - loss: 1.3563 - accuracy: 0.3365 - val_loss: 1.3827 - val_accuracy: 0.2827
Epoch 7/20
125/125 [==============================] - 4s 31ms/step - loss: 1.3437 - accuracy: 0.3494 - val_loss: 1.3851 - val_accuracy: 0.2958

In [ ]:
model_cbow = tf.keras.Sequential()
model_cbow.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow.add(tf.keras.layers.LSTM(50))
model_cbow.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow.fit(X_train_cbow,y_train_cbow,epochs=20,validation_split=0.2)
history = model_cbow.evaluate(X_test_cbow,y_test_cbow)
print(history)

result_table+=[history[1]]
print("CBOW_LSTM")

Epoch 1/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3733 - accuracy: 0.3260 - val_loss: 1.3920 - val_accuracy: 0.2978
Epoch 2/20
125/125 [==============================] - 1s 6ms/step - loss: 1.3706 - accuracy: 0.3272 - val_loss: 1.3887 - val_accuracy: 0.2978
Epoch 3/20
125/125 [==============================] - 1s 5ms/step - loss: 1.3706 - accuracy: 0.3272 - val_loss: 1.3857 - val_accuracy: 0.2978
Epoch 4/20
125/125 [==============================] - 1s 6ms/step - loss: 1.3706 - accuracy: 0.3272 - val_loss: 1.3820 - val_accuracy: 0.2978
Epoch 5/20
125/125 [==============================] - 1s 5ms/step - loss: 1.3704 - accuracy: 0.3272 - val_loss: 1.3846 - val_accuracy: 0.2978
Epoch 6/20
125/125 [==============================] - 1s 5ms/step - loss: 1.3706 - accuracy: 0.3272 - val_loss: 1.3845 - val_accuracy: 0.2978
Epoch 7/20
125/125 [==============================] - 1s 5ms/step - loss: 1.3702 - accuracy: 0.3272 - val_loss: 1.3838 - val_accuracy: 0.2978
Epoch 

In [ ]:
model_cbow_bi = tf.keras.Sequential()
model_cbow_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_cbow_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow_bi.fit(X_train_cbow,y_train_cbow,epochs=20,validation_split=0.2)
history = model_cbow_bi.evaluate(X_test_cbow,y_test_cbow)
print(history)

result_table+=[history[1]]
print("CBOW_BI")

Epoch 1/20
125/125 [==============================] - 2s 13ms/step - loss: 1.3720 - accuracy: 0.3247 - val_loss: 1.3852 - val_accuracy: 0.2978
Epoch 2/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3708 - accuracy: 0.3272 - val_loss: 1.3845 - val_accuracy: 0.2978
Epoch 3/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3704 - accuracy: 0.3272 - val_loss: 1.3860 - val_accuracy: 0.2978
Epoch 4/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3696 - accuracy: 0.3272 - val_loss: 1.3843 - val_accuracy: 0.2978
Epoch 5/20
125/125 [==============================] - 1s 8ms/step - loss: 1.3694 - accuracy: 0.3272 - val_loss: 1.3822 - val_accuracy: 0.2978
Epoch 6/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3683 - accuracy: 0.3272 - val_loss: 1.3829 - val_accuracy: 0.2978
Epoch 7/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3680 - accuracy: 0.3272 - val_loss: 1.3822 - val_accuracy: 0.2978
Epoch

In [ ]:
model_cbow_rnn = tf.keras.Sequential()
model_cbow_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_rnn.add(tf.keras.layers.SimpleRNN(50))
model_cbow_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow_rnn.fit(X_train_cbow,y_train_cbow,epochs=20,validation_split=0.2)
history = model_cbow_rnn.evaluate(X_test_cbow,y_test_cbow)
print(history)

result_table+=[history[1]]
print("CBOW_RNN")

Epoch 1/20
125/125 [==============================] - 4s 32ms/step - loss: 1.3747 - accuracy: 0.3176 - val_loss: 1.3882 - val_accuracy: 0.2978
Epoch 2/20
125/125 [==============================] - 4s 32ms/step - loss: 1.3710 - accuracy: 0.3275 - val_loss: 1.3888 - val_accuracy: 0.2978
Epoch 3/20
125/125 [==============================] - 4s 30ms/step - loss: 1.3706 - accuracy: 0.3239 - val_loss: 1.3924 - val_accuracy: 0.2978
Epoch 4/20
125/125 [==============================] - 4s 31ms/step - loss: 1.3694 - accuracy: 0.3270 - val_loss: 1.3856 - val_accuracy: 0.3008
Epoch 5/20
125/125 [==============================] - 4s 32ms/step - loss: 1.3694 - accuracy: 0.3300 - val_loss: 1.3838 - val_accuracy: 0.2847
Epoch 6/20
125/125 [==============================] - 4s 32ms/step - loss: 1.3707 - accuracy: 0.3295 - val_loss: 1.3831 - val_accuracy: 0.2978
Epoch 7/20
125/125 [==============================] - 4s 32ms/step - loss: 1.3680 - accuracy: 0.3272 - val_loss: 1.3890 - val_accuracy: 0.2978

In [ ]:
model_glove = tf.keras.Sequential()
model_glove.add(tf.keras.Input(shape=(Max_input_size_glove,100)))
model_glove.add(tf.keras.layers.LSTM(50))
model_glove.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove.fit(X_train_glove,y_train_glove,epochs=20,validation_split=0.2)
history = model_glove.evaluate(X_test_glove,y_test_glove)
print(history)

result_table+=[history[1]]
print("GLOVE_LSTM")

Epoch 1/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3458 - accuracy: 0.3413 - val_loss: 1.3344 - val_accuracy: 0.4074
Epoch 2/20
125/125 [==============================] - 1s 5ms/step - loss: 1.1914 - accuracy: 0.4604 - val_loss: 1.1659 - val_accuracy: 0.4336
Epoch 3/20
125/125 [==============================] - 1s 6ms/step - loss: 1.0888 - accuracy: 0.4908 - val_loss: 1.1302 - val_accuracy: 0.4477
Epoch 4/20
125/125 [==============================] - 1s 6ms/step - loss: 1.0188 - accuracy: 0.5293 - val_loss: 1.0865 - val_accuracy: 0.5121
Epoch 5/20
125/125 [==============================] - 1s 5ms/step - loss: 0.9532 - accuracy: 0.5696 - val_loss: 1.0410 - val_accuracy: 0.5513
Epoch 6/20
125/125 [==============================] - 1s 5ms/step - loss: 0.8754 - accuracy: 0.6167 - val_loss: 1.0373 - val_accuracy: 0.5905
Epoch 7/20
125/125 [==============================] - 1s 6ms/step - loss: 0.8234 - accuracy: 0.6547 - val_loss: 0.9845 - val_accuracy: 0.5946
Epoch 

In [ ]:
model_glove_bi = tf.keras.Sequential()
model_glove_bi.add(tf.keras.Input(shape=(Max_input_size_glove,100)))
model_glove_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_glove_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_bi.fit(X_train_glove,y_train_glove,epochs=20,validation_split=0.2)
history = model_glove_bi.evaluate(X_test_glove,y_test_glove)
print(history)

result_table+=[history[1]]
print("GLOVE_BI")

Epoch 1/20
125/125 [==============================] - 2s 13ms/step - loss: 1.2328 - accuracy: 0.4369 - val_loss: 1.0911 - val_accuracy: 0.5332
Epoch 2/20
125/125 [==============================] - 1s 9ms/step - loss: 0.9896 - accuracy: 0.5892 - val_loss: 0.9593 - val_accuracy: 0.6076
Epoch 3/20
125/125 [==============================] - 1s 9ms/step - loss: 0.8354 - accuracy: 0.6700 - val_loss: 0.8531 - val_accuracy: 0.6630
Epoch 4/20
125/125 [==============================] - 1s 9ms/step - loss: 0.7206 - accuracy: 0.7196 - val_loss: 0.7797 - val_accuracy: 0.6972
Epoch 5/20
125/125 [==============================] - 1s 9ms/step - loss: 0.6233 - accuracy: 0.7586 - val_loss: 0.7616 - val_accuracy: 0.7173
Epoch 6/20
125/125 [==============================] - 1s 8ms/step - loss: 0.5366 - accuracy: 0.7931 - val_loss: 0.7404 - val_accuracy: 0.7193
Epoch 7/20
125/125 [==============================] - 1s 9ms/step - loss: 0.4649 - accuracy: 0.8210 - val_loss: 0.7489 - val_accuracy: 0.7354
Epoch

In [ ]:
model_glove_rnn = tf.keras.Sequential()
model_glove_rnn.add(tf.keras.Input(shape=(Max_input_size_glove,100)))
model_glove_rnn.add(tf.keras.layers.SimpleRNN(50))
model_glove_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_rnn.fit(X_train_glove,y_train_glove,epochs=20,validation_split=0.2)
history = model_glove_rnn.evaluate(X_test_glove,y_test_glove)
print(history)

result_table+=[history[1]]
print("GLOVE_RNN")

Epoch 1/20
125/125 [==============================] - 4s 32ms/step - loss: 1.3430 - accuracy: 0.3521 - val_loss: 1.3098 - val_accuracy: 0.4044
Epoch 2/20
125/125 [==============================] - 4s 33ms/step - loss: 1.3003 - accuracy: 0.4168 - val_loss: 1.2913 - val_accuracy: 0.4034
Epoch 3/20
125/125 [==============================] - 4s 30ms/step - loss: 1.2796 - accuracy: 0.4284 - val_loss: 1.2860 - val_accuracy: 0.4125
Epoch 4/20
125/125 [==============================] - 4s 30ms/step - loss: 1.3316 - accuracy: 0.3740 - val_loss: 1.3438 - val_accuracy: 0.3571
Epoch 5/20
125/125 [==============================] - 4s 31ms/step - loss: 1.2996 - accuracy: 0.4022 - val_loss: 1.3032 - val_accuracy: 0.3994
Epoch 6/20
125/125 [==============================] - 4s 32ms/step - loss: 1.3252 - accuracy: 0.3896 - val_loss: 1.3766 - val_accuracy: 0.2978
Epoch 7/20
125/125 [==============================] - 4s 30ms/step - loss: 1.3439 - accuracy: 0.3662 - val_loss: 1.3593 - val_accuracy: 0.3159

In [ ]:
print("\tLstm\t\t\tBiLstm\t\t\tRnn")
print("sg",result_table[:3])
print("cbow",result_table[3:6])
print("glove",result_table[6:9])

	Lstm			BiLstm			Rnn
sg [0.3079812228679657, 0.3417840301990509, 0.2694835662841797]
cbow [0.3079812228679657, 0.3272300362586975, 0.30516430735588074]
glove [0.7295774817466736, 0.7403755784034729, 0.41455399990081787]
